In [1]:
import os
from tqdm.auto import tqdm
import joblib
import pickle
from sklearn.metrics import classification_report
import numpy as np

In [8]:
import keras

In [9]:
languages = ['english_1','filipino','chinese','korean']

### Save All Tradition models in Output Folder in Joblib Form

In [10]:
os.listdir('output')

['filipino_SVM_char.joblib',
 'korean_RF_char.joblib',
 'chinese_SVM_word.joblib',
 '.DS_Store',
 'english_1_GBT_word.joblib',
 'korean_GBT_char.joblib',
 'english_1_RF_word.joblib',
 'filipino_NB_char.joblib',
 'chinese_NB_word.joblib',
 'filipino_LR_word.joblib',
 'chinese_LR_char.joblib',
 'korean_NB_char.joblib',
 'korean_LR_word.joblib',
 'chinese_GBT_char.joblib',
 'filipino_GBT_word.joblib',
 'english_1_NB_word.joblib',
 'chinese_RF_word.joblib',
 'filipino_RF_char.joblib',
 'english_1_SVM_char.joblib',
 'korean_SVM_word.joblib',
 'english_1_LR_char.joblib',
 'chinese_LR_word.joblib',
 'filipino_LR_char.joblib',
 'english_1_RF_char.joblib',
 'chinese_NB_char.joblib',
 'filipino_NB_word.joblib',
 'english_1_GBT_char.joblib',
 'korean_GBT_word.joblib',
 'chinese_SVM_char.joblib',
 'filipino_SVM_word.joblib',
 'korean_RF_word.joblib',
 'english_1_LR_word.joblib',
 'english_1_SVM_word.joblib',
 'english_1_NB_char.joblib',
 'filipino_RF_word.joblib',
 'chinese_RF_char.joblib',
 'kore

### Save all CNN Models in temp as 

In [11]:
os.listdir('temp/models')

['english_1_model_char',
 'korean_model_word',
 'filipino_model_char',
 '.DS_Store',
 'chinese_model_char',
 'chinese_model_hybrid',
 'korean_model_GLOVE_word',
 'english_1_model_GLOVE_word',
 'chinese_model_GLOVE_word',
 'filipino_model_hybrid',
 'korean_model_GLOVE_hybrid',
 'korean_model_hybrid',
 'chinese_model_GLOVE_hybrid',
 'english_1_model_word',
 'korean_model_char',
 'english_1_model_GLOVE_hybrid',
 'korean_model_GLOVE_char',
 'english_1_model_GLOVE_char',
 'filipino_model_word',
 'chinese_model_word',
 'chinese_model_GLOVE_char',
 'english_1_model_hybrid']

### Save Embeddings in temp folder of this notebook as

In [12]:
os.listdir('temp/embeds')

['korean',
 '.DS_Store',
 'english_1',
 'english_1_glove',
 'chinese',
 'filipino',
 'chinese_glove',
 'korean_glove']

# All Models

In [13]:

def vector2matrix(text_vector, max_len=140, N_DIM=70):
	matrix = np.zeros((max_len, N_DIM))
	for i, index_elem in enumerate(text_vector):
		row = np.zeros(N_DIM)
		if int(index_elem) != -1:
			row[int(index_elem)] = 1
		matrix[i] = row
	return matrix

## Maximize Probability

In [14]:
for lang in languages:
    print(lang)
    models = [i for i in os.listdir('output') if lang in i]
    proc_data = pickle.load(open(f'processed_data/{lang}_processed.pkl','rb'))
    preds = []
    for m in tqdm(models):
        loaded_model = joblib.load(f'output/{m}')
        preds.append(loaded_model[0].predict_proba(proc_data['text_test']))
        
        
        
    X_word = np.load(f'temp/embeds/{lang}/CtxtText_InputText_test.npy')[:,100:]
    # X_char = np.load(f'temp/embeds/{lang}//Char_InputText_test.npy')
    char_text_data_list = []
    _char_text_data = np.load(f'temp/embeds/{lang}//Char_InputText_test.npy')
    for text_vector in _char_text_data:
        char_text_data_list.append(vector2matrix(text_vector))
    char_text_data = np.asarray(char_text_data_list)
    X_char = char_text_data

#     preds = []

    for cnn_model in tqdm([i for i in os.listdir('temp/models') if lang in i]):
        model = keras.models.load_model(f'temp/models/{cnn_model}')
        if 'char' in cnn_model:
            preds.append(model.predict(X_char,batch_size=128))
        elif 'word' in cnn_model:
            preds.append(model.predict(X_word,batch_size=128))
        else:
            preds.append(model.predict([X_word,X_char],batch_size=128))

        
    ans = []
    for i in range(len(preds[0])):
        ts = []
        for j in range(len(preds)):
            ts = ts + list(preds[j][i])
        ans.append(ts.index(max(ts))%2)
    test_label_data = [i.tolist().index(1) for i in proc_data['label_test']]
    print(classification_report(test_label_data, ans,target_names=list(proc_data['label_dict'].keys())))


english_1


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

2021-12-02 19:06:00.739228: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-02 19:06:00.742210: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


               precision    recall  f1-score   support

    offensive       0.93      0.36      0.52      4124
not_offensive       0.21      0.86      0.34       833

     accuracy                           0.45      4957
    macro avg       0.57      0.61      0.43      4957
 weighted avg       0.81      0.45      0.49      4957

filipino


...

  0%|          | 0/6 [00:00<?, ?it/s]

               precision    recall  f1-score   support

    offensive       0.35      0.68      0.46       619
not_offensive       0.66      0.33      0.44      1175

     accuracy                           0.45      1794
    macro avg       0.50      0.50      0.45      1794
 weighted avg       0.55      0.45      0.45      1794

korean


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

               precision    recall  f1-score   support

not_offensive       0.45      0.54      0.49       697
    offensive       0.58      0.50      0.54       903

     accuracy                           0.52      1600
    macro avg       0.52      0.52      0.52      1600
 weighted avg       0.53      0.52      0.52      1600



## Voting

In [15]:
for lang in languages:
    print(lang)
    models = [i for i in os.listdir('output') if lang in i]
    proc_data = pickle.load(open(f'processed_data/{lang}_processed.pkl','rb'))
    preds = []
    for m in tqdm(models):
        loaded_model = joblib.load(f'output/{m}')
        preds.append(loaded_model[0].predict_proba(proc_data['text_test']))
        
        
    X_word = np.load(f'temp/embeds/{lang}/CtxtText_InputText_test.npy')[:,100:]
    # X_char = np.load(f'temp/embeds/{lang}//Char_InputText_test.npy')
    char_text_data_list = []
    _char_text_data = np.load(f'temp/embeds/{lang}//Char_InputText_test.npy')
    for text_vector in _char_text_data:
        char_text_data_list.append(vector2matrix(text_vector))
    char_text_data = np.asarray(char_text_data_list)
    X_char = char_text_data

#     preds = []

    for cnn_model in tqdm([i for i in os.listdir('temp/models') if lang in i]):
        model = keras.models.load_model(f'temp/models/{cnn_model}')
        if 'char' in cnn_model:
            preds.append(model.predict(X_char,batch_size=128))
        elif 'word' in cnn_model:
            preds.append(model.predict(X_word,batch_size=128))
        else:
            preds.append(model.predict([X_word,X_char],batch_size=128))
    if lang=='filipino':
        level = 7
    else:
        level = 8
    ans = []
    for i in range(len(preds[0])):
        ts = []
        for j in range(len(preds)):
            ts = ts + list([preds[j][i].argmax()])
        ans.append(1 if np.count_nonzero(ts)>=level else 0)
    test_label_data = [i.tolist().index(1) for i in proc_data['label_test']]
    print(classification_report(test_label_data, ans,target_names=list(proc_data['label_dict'].keys())))


english_1


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

               precision    recall  f1-score   support

    offensive       0.96      0.97      0.96      4124
not_offensive       0.85      0.78      0.81       833

     accuracy                           0.94      4957
    macro avg       0.90      0.88      0.89      4957
 weighted avg       0.94      0.94      0.94      4957

filipino


  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

               precision    recall  f1-score   support

    offensive       0.85      0.22      0.34       619
not_offensive       0.70      0.98      0.82      1175

     accuracy                           0.72      1794
    macro avg       0.78      0.60      0.58      1794
 weighted avg       0.75      0.72      0.66      1794

korean


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

               precision    recall  f1-score   support

not_offensive       0.80      0.41      0.54       697
    offensive       0.67      0.92      0.77       903

     accuracy                           0.70      1600
    macro avg       0.73      0.66      0.66      1600
 weighted avg       0.73      0.70      0.67      1600



# Word Models

## Maximize Probability

In [16]:
for lang in languages:
    print(lang)
    models = [i for i in os.listdir('output') if lang in i and 'word' in i]
    proc_data = pickle.load(open(f'processed_data/{lang}_processed.pkl','rb'))
    preds = []
    for m in tqdm(models):
        loaded_model = joblib.load(f'output/{m}')
        preds.append(loaded_model[0].predict_proba(proc_data['text_test']))
        
        
        
    X_word = np.load(f'temp/embeds/{lang}/CtxtText_InputText_test.npy')[:,100:]
    # X_char = np.load(f'temp/embeds/{lang}//Char_InputText_test.npy')
    char_text_data_list = []
    _char_text_data = np.load(f'temp/embeds/{lang}//Char_InputText_test.npy')
    for text_vector in _char_text_data:
        char_text_data_list.append(vector2matrix(text_vector))
    char_text_data = np.asarray(char_text_data_list)
    X_char = char_text_data

#     preds = []

    for cnn_model in tqdm([i for i in os.listdir('temp/models') if lang in i and 'word' in i]):
        model = keras.models.load_model(f'temp/models/{cnn_model}')
        if 'char' in cnn_model:
            preds.append(model.predict(X_char,batch_size=128))
        elif 'word' in cnn_model:
            preds.append(model.predict(X_word,batch_size=128))
        else:
            preds.append(model.predict([X_word,X_char],batch_size=128))

        
    ans = []
    for i in range(len(preds[0])):
        ts = []
        for j in range(len(preds)):
            ts = ts + list(preds[j][i])
        ans.append(ts.index(max(ts))%2)
    test_label_data = [i.tolist().index(1) for i in proc_data['label_test']]
    print(classification_report(test_label_data, ans,target_names=list(proc_data['label_dict'].keys())))


english_1


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

               precision    recall  f1-score   support

    offensive       0.92      0.31      0.46      4124
not_offensive       0.20      0.87      0.33       833

     accuracy                           0.40      4957
    macro avg       0.56      0.59      0.39      4957
 weighted avg       0.80      0.40      0.44      4957

filipino


  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

               precision    recall  f1-score   support

    offensive       0.35      0.67      0.46       619
not_offensive       0.66      0.33      0.44      1175

     accuracy                           0.45      1794
    macro avg       0.50      0.50      0.45      1794
 weighted avg       0.55      0.45      0.45      1794

korean


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

               precision    recall  f1-score   support

not_offensive       0.46      0.48      0.47       697
    offensive       0.58      0.55      0.57       903

     accuracy                           0.52      1600
    macro avg       0.52      0.52      0.52      1600
 weighted avg       0.53      0.52      0.52      1600



## Voting

In [17]:
for lang in languages:
    print(lang)
    models = [i for i in os.listdir('output') if lang in i and 'word' in i]
    proc_data = pickle.load(open(f'processed_data/{lang}_processed.pkl','rb'))
    preds = []
    for m in tqdm(models):
        loaded_model = joblib.load(f'output/{m}')
        preds.append(loaded_model[0].predict_proba(proc_data['text_test']))
        
        
    X_word = np.load(f'temp/embeds/{lang}/CtxtText_InputText_test.npy')[:,100:]
    # X_char = np.load(f'temp/embeds/{lang}//Char_InputText_test.npy')
    char_text_data_list = []
    _char_text_data = np.load(f'temp/embeds/{lang}//Char_InputText_test.npy')
    for text_vector in _char_text_data:
        char_text_data_list.append(vector2matrix(text_vector))
    char_text_data = np.asarray(char_text_data_list)
    X_char = char_text_data

#     preds = []

    for cnn_model in tqdm([i for i in os.listdir('temp/models') if lang in i and 'word' in i]):
        model = keras.models.load_model(f'temp/models/{cnn_model}')
        if 'char' in cnn_model:
            preds.append(model.predict(X_char,batch_size=128))
        elif 'word' in cnn_model:
            preds.append(model.predict(X_word,batch_size=128))
        else:
            preds.append(model.predict([X_word,X_char],batch_size=128))
            
    if lang=='filipino':
        level = 4
    else:
        level = 4
    ans = []
    for i in range(len(preds[0])):
        ts = []
        for j in range(len(preds)):
            ts = ts + list([preds[j][i].argmax()])
        ans.append(1 if np.count_nonzero(ts)>=level else 0)
    print(len(ts))
    test_label_data = [i.tolist().index(1) for i in proc_data['label_test']]
    print(classification_report(test_label_data, ans,target_names=list(proc_data['label_dict'].keys())))


english_1


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

7
               precision    recall  f1-score   support

    offensive       0.94      0.99      0.96      4124
not_offensive       0.91      0.67      0.78       833

     accuracy                           0.93      4957
    macro avg       0.93      0.83      0.87      4957
 weighted avg       0.93      0.93      0.93      4957

filipino


  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

7
               precision    recall  f1-score   support

    offensive       0.68      0.03      0.06       619
not_offensive       0.66      0.99      0.79      1175

     accuracy                           0.66      1794
    macro avg       0.67      0.51      0.43      1794
 weighted avg       0.67      0.66      0.54      1794

korean


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

7
               precision    recall  f1-score   support

not_offensive       0.69      0.45      0.54       697
    offensive       0.66      0.84      0.74       903

     accuracy                           0.67      1600
    macro avg       0.68      0.65      0.64      1600
 weighted avg       0.67      0.67      0.66      1600



# Char Models

## Maximize Probability

In [18]:
for lang in languages:
    print(lang)
    models = [i for i in os.listdir('output') if lang in i and 'char' in i]
    proc_data = pickle.load(open(f'processed_data/{lang}_processed.pkl','rb'))
    preds = []
    for m in tqdm(models):
        loaded_model = joblib.load(f'output/{m}')
        preds.append(loaded_model[0].predict_proba(proc_data['text_test']))
        
        
        
    X_word = np.load(f'temp/embeds/{lang}/CtxtText_InputText_test.npy')[:,100:]
    # X_char = np.load(f'temp/embeds/{lang}//Char_InputText_test.npy')
    char_text_data_list = []
    _char_text_data = np.load(f'temp/embeds/{lang}//Char_InputText_test.npy')
    for text_vector in _char_text_data:
        char_text_data_list.append(vector2matrix(text_vector))
    char_text_data = np.asarray(char_text_data_list)
    X_char = char_text_data

#     preds = []

    for cnn_model in tqdm([i for i in os.listdir('temp/models') if lang in i and 'char' in i]):
        model = keras.models.load_model(f'temp/models/{cnn_model}')
        if 'char' in cnn_model:
            preds.append(model.predict(X_char,batch_size=128))
        elif 'word' in cnn_model:
            preds.append(model.predict(X_word,batch_size=128))
        else:
            preds.append(model.predict([X_word,X_char],batch_size=128))

        
    ans = []
    for i in range(len(preds[0])):
        ts = []
        for j in range(len(preds)):
            ts = ts + list(preds[j][i])
        ans.append(ts.index(max(ts))%2)
    test_label_data = [i.tolist().index(1) for i in proc_data['label_test']]
    print(classification_report(test_label_data, ans,target_names=list(proc_data['label_dict'].keys())))


english_1


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

               precision    recall  f1-score   support

    offensive       0.93      0.50      0.65      4124
not_offensive       0.25      0.82      0.38       833

     accuracy                           0.55      4957
    macro avg       0.59      0.66      0.51      4957
 weighted avg       0.82      0.55      0.60      4957

filipino


  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

               precision    recall  f1-score   support

    offensive       0.73      0.56      0.63       619
not_offensive       0.79      0.89      0.84      1175

     accuracy                           0.78      1794
    macro avg       0.76      0.72      0.74      1794
 weighted avg       0.77      0.78      0.77      1794

korean


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

               precision    recall  f1-score   support

not_offensive       0.76      0.44      0.55       697
    offensive       0.67      0.89      0.77       903

     accuracy                           0.69      1600
    macro avg       0.71      0.66      0.66      1600
 weighted avg       0.71      0.69      0.67      1600



## Voting

In [19]:
for lang in languages:
    print(lang)
    models = [i for i in os.listdir('output') if lang in i and 'char' in i]
    proc_data = pickle.load(open(f'processed_data/{lang}_processed.pkl','rb'))
    preds = []
    for m in tqdm(models):
        loaded_model = joblib.load(f'output/{m}')
        preds.append(loaded_model[0].predict_proba(proc_data['text_test']))
        
        
    X_word = np.load(f'temp/embeds/{lang}/CtxtText_InputText_test.npy')[:,100:]
    # X_char = np.load(f'temp/embeds/{lang}//Char_InputText_test.npy')
    char_text_data_list = []
    _char_text_data = np.load(f'temp/embeds/{lang}//Char_InputText_test.npy')
    for text_vector in _char_text_data:
        char_text_data_list.append(vector2matrix(text_vector))
    char_text_data = np.asarray(char_text_data_list)
    X_char = char_text_data


    for cnn_model in tqdm([i for i in os.listdir('temp/models') if lang in i and 'char' in i]):
        model = keras.models.load_model(f'temp/models/{cnn_model}')
        if 'char' in cnn_model:
            preds.append(model.predict(X_char,batch_size=128))
        elif 'word' in cnn_model:
            preds.append(model.predict(X_word,batch_size=128))
        else:
            preds.append(model.predict([X_word,X_char],batch_size=128))
    if lang=='filipino':
        level = 4
    else:
        level = 4
    ans = []
    for i in range(len(preds[0])):
        ts = []
        for j in range(len(preds)):
            ts = ts + list([preds[j][i].argmax()])
        ans.append(1 if np.count_nonzero(ts)>=level else 0)
    print(len(ts))
    test_label_data = [i.tolist().index(1) for i in proc_data['label_test']]
    print(classification_report(test_label_data, ans,target_names=list(proc_data['label_dict'].keys())))


english_1


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

7
               precision    recall  f1-score   support

    offensive       0.94      0.98      0.96      4124
not_offensive       0.88      0.69      0.77       833

     accuracy                           0.93      4957
    macro avg       0.91      0.84      0.87      4957
 weighted avg       0.93      0.93      0.93      4957

filipino


  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

7
               precision    recall  f1-score   support

    offensive       0.87      0.36      0.51       619
not_offensive       0.74      0.97      0.84      1175

     accuracy                           0.76      1794
    macro avg       0.81      0.67      0.68      1794
 weighted avg       0.79      0.76      0.73      1794

korean


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

7
               precision    recall  f1-score   support

not_offensive       0.79      0.47      0.59       697
    offensive       0.69      0.90      0.78       903

     accuracy                           0.71      1600
    macro avg       0.74      0.69      0.68      1600
 weighted avg       0.73      0.71      0.70      1600

